# Project Report
#### Magdalena Barros, Paula Cadena, Michael Rosenbaum | CAPP 30254: Machine Learning for Public Policy 
#### Code available on [GitHub](https://github.com/m-rosenbaum/cr_pmt)

## 0 | Abstract

We design a data flow to implement a Proxy Means Test (PMT) to classify Costa Rican households into four categories of poverty. These types of tests are common in international development, where readily available information on household composition and assets can proxy income to target social programs ([Grosh & Baker, 2013](https://elibrary.worldbank.org/doi/abs/10.1596/0-8213-3313-5)). This work uses a [Kaggle Dataset](https://www.kaggle.com/competitions/costa-rican-household-poverty-prediction/data), including partially cleaned data from the Inter-American Development Bank (IDB) on 2,988 Costa Rican households.

This data is highly imbalanced and includes several repeated and conditional measures. Before tuning candidate models, we first reduce the dataset to a foundational set of durable assets, household composition, and housing features and then use synthetic oversampling techniques to create a balanced dataset on our poverty labels.

Based on a grid search of hyperparameters across penalized multinomial regressions, random forests, and KNN, we selected random forest to predict poverty status. We achieved an "F1 macro" score of 37% predictive power on our test sample measured by F1 scores, which average the F1 score of each label. This corresponds to 62% overall accuracy due to the sample imbalance. Although our final model evaluation includes recall as a component of the "F1 macro" score, the model tends to predict false negatives for the higher poverty categories at higher rates. It also performs worse than the winning Kaggle scores, which have approximately 45% accuracy measured by "F1 macro" scores.

## 1 | Data Set-up

Our data exploration has focused on understanding the underlying dataset. The data comes from a [2017 ILO survey administered](https://webapps.ilo.org/surveyLib/index.php/catalog/7230/related-materials) by the IDB. It is drawn from a nationally-representative household survey and includes a subset of household— and individual-level variables.

The initial Kaggle competition was meant to provide the IDB with different approaches to applying PMTs to social programs in low—and medium-income countries. As such, the data provided is relatively clean compared to the original survey data, but it also includes a number of variables that would not be useful for prediction, are created by the IDB, or would introduce challenges to claim a classifier. In our data preparation, we handle three types of specific characteristics: missingness, label imbalance, and repeated measures.

### 1.1 | Missingness

Three variables have a significant amount of missing values:

1. Type of house (`v2a1`): We replace NaN values with zero in cases where houses are fully paid or non-rent-paying and retain rent status in each mode.
1. Number of tablets owned (`v18q1`): We combine the indicator variable for owning a tablet with the number of tablets owned to produce a non-missing variable of number of tablets owned.  
1. Grades behind (`rez_esc`): We do not include this in the household-level model we predict from but leave it in the individual-level dataset to produce features as needed.

### 1.2 | Outcome Distribution

The label distribution is also heavily skewed. Even when the label is subset to only one observation per household, nearly 70% of the observations are in the highest income category.

<a href="url"><img src='../assets/label_skew.png' align="center" height = '50%'></a>

As we intend to use some classifiers, such as KNN, that would require weighting to produce accurate classification measures in minority categories, we introduce synthetic oversampling to create a training set balanced on panels. To do so, we follow this procedure:

1. Split the sample into 80/20 percent test/train split.
2. Standardize continuous variables to be mean-centered with a standard deviation of 1.
3. Oversample the outcome variables within the cross-validation process.

Future efforts could include different approaches to weighting and oversampling.

### 1.3 | Repeated measures

We remove several features based on repeated measurements of the same underlying construct or the distribution of the outcome variable.

- Drop features calculated by : `edjefe`, `edjefa`, `area2`, `r4h1`, `r4h2`, `r4h3`, `r4m1`, `r4m2`, `r4m3`, `r4t1`, `r4t2`, `overcrowding`, `tamhog`, `tamviv`, `male`, `hogar_total`, `dependency`, `meaneduc`, `SQBescolari`, `SQBage`, `SQBhogar_total`, `SQBedjefe`, `SQBhogar_nin`, `SQBovercrowding`, `SQBdependency`, `SQBmeaned`, `agesq`, `techozinc`, `techoentrepiso`, `techocane`, `techootro`.
- Retain `hhsize` and drop `tamhog`, `r4t3`, `tamviv` as `hhsize` is calculated by the survey software based on the household roster, whereas each other is not aligned with the counts of household compositions variables (age, sex, etc.) 
- Collapse individual-level categorical variables with percentages less than 5%: `estadocivil`, `instlevel`, and `parentesco`. 5% is chosen arbitrarily as a small proportion of the dataset.
- Collapse household-level categorical variables with percentages less than 5%, including durable assets: `piso`, `pared`, `techo`, `sanitario`, `elimbasu`, `tipovivi`. 5% is chosen arbitrarily as a small proportion of the dataset.

After completing the data setup, we are left with 82 features in our "foundational dataset."

### 1.4 | Final Data Preparation Pipeline

Our final pipeline can be summarized as follows:

1. Clean the survey data into a standardized format, based on content-agnostic decisions.

    1.1 Clean missing values.

    1.2 Clean individual-level categorical variables based.
    
    1.3 Clean household-level categorical variables. 

    1.4 Create features as needed.

    1.5 Drop extraneous variables.

    1.6 Collapse to the household-level on `idhogar`.
    
    1.7 Standardize all continuous variables to be mean-centered with standard deviation of 1.

2. Conduct an 80 / 20 test-train split**. 

3. Synthetic oversampling of data prior to cross-validation. 

Note that we do not include the  70 / 20 / 10 test-train-validation split in the project description. This choice was based on our use of cross-validation. This process is doing the sample splitting, making the validation set not necessary ([scikit-learn Cross-Validation](https://scikit-learn.org/stable/modules/cross_validation.html#:~:text=A%20test%20set%20should%20still,generally%20follow%20the%20same%20principles)).

## 3 | Model Tuning

We tune three types of models based on the content covered in this course, with the aim of applying each of them in the PMT context and exploring a variety of hyperparameter estimates. This section first summarizes each of the models used, including why they may apply to this task, and then interprets the cross-validation procedure we conduct.

### 3.1 | Model choice

The Costa Rican PMT context requires classification into four ordinal classes. This limits the set of models we can include slightly, but we still focus on the key models discussed in the course outside of Neural Nets and Perceptrons:

1. **K-Nearest Neighbors** (12): Class labels are assigned based on a vote from *K* neighbors across a given distance function. As our features are standardized and neighbors normalized, we do not weigh different features differently. Instead, we explore two sets of hyperparameters across 12 different models:
    1. *K* (number of nearest neighbors): We explore different neighborhood sizes, including {3, 5, 10, 25, 50, 100}. We expect larger leaves to perform better on the test data due to the variance in the data and the risk of overfitting on smaller neighborhood sizes.
    1. *Distance weighting*: We vary between unweighted and weighted.
2. **Penalized multinomial regression** (16): The scikit-learn logistic regression classifier allows us to estimate multinomial logit models for multiple categories. This gives us probabilities of any input observation in any of our labels, with the highest probability preferred. Our primary concern was estimation: we used the `liblinear` solver in `scikit-learn` as it can solve both $L_1$ and $L_2$ penalties and was both quicker and more likely to converge across our hyperparameter tuples in our testing.
    1. *Penalty form*: We consider both $L_1$ and $L_2$ penalties to handle the survey data's sparseness appropriately. Most variables are indicator variables for multiple-response option variables. 
    1. *$\lambda$*: We vary the penalty amount over powers of 10 to explore a large space of penalty terms, and to encourage a range of parsimonious and complex models. We cover the following penalty values: {0.01, 0.1, 1, 10, 100, 1000, 10000, 100000}.
3. **Random forests** (144): We estimate a variety of random forest models using `scikit-learn`'s default classifier across an array of hyperparameters. As with the other models, we are concerned about overfitting sparse and noisy data. We focus primarily on controlling how parsimonious each tree is and how inconsistent average trees are in the forest to push towards a simple model given the variance in the input data. We choose a Gini split criterion based on research suggesting that Gini is quicker and is unlikely to matter for the accuracy of prediction ([Raileanu & Stoffel, 2004](https://link.springer.com/article/10.1023/b:amai.0000018580.96245.c6)). Due to the number and plausible range of the hyperparameters for random forests, we only include a small candidate set that aims to explore across a larger hyperparameter space but likely misses the most efficient area.
    1. *Number of trees*: We use forests with trees of size {25, 50, 100}
    1. *Maximum depth of trees*: We allow trees to grow to full depth or keep them relatively short, only allowing five splits (32 leaves averaging 65 observations). We test the following values: {100% features (82), 50% features (41), 25% features (20), five features}
    1. *Minimum observations per leaf*: We allow the trees to end on leaves with relatively small proportions of the sample included, ranging from a single observation to 50: {1, 5, 10, 50}.
    1. *Percent of observations sampled*: We include various resampling rates, including between 25% and 75% {0.25, 0.50, 0.75}

This results in 172 models to estimate. If this were a larger project, we would consider more models, including a Neural Net, and more expansive searches across the hyperparameter space. In addition, we did not discuss multiclass perceptrons and therefore do not implement them in this project. 

### 3.2 | Cross-validation

To evaluate our model's performance, we use K-fold cross-validation with 5-folds. This involves repeatedly splitting our training data into 80/20 train-validation splits, evaluating the model's accuracy, and then averaging the estimates across the five attempts. 

As mentioned above, we oversampled our data to predict the minority categories better. We use the SMOTE method to do so for two reasons: large enough N in the minority classes to produce a sufficient spread of synthetic data and ease of implementation in Python ([Chawla et al., 2002](https://doi.org/10.1613/jair.953)). Since we oversample our data using synthetic data, we follow [Damien Martin's procedure](https://kiwidamien.github.io/how-to-do-cross-validation-when-upsampling-data.html) to avoid any data leakage and therefore bias in our training set estimates. We evaluated the model's accuracy on "F1 Macro" scores, which is the unweighted average of the precision and recall scores for each class. Compared to overall prediction accuracy, we wanted to focus on avoiding false negatives due to the policy context without only optimizing for recall. As discussed above, false negatives would harm households receiving benefits and are likely worse for welfare than false positives in this context.

Since we've kept the candidate set of models at a reasonable amount, we're able to conduct a grid search over all combinations of models and parameters to select our best performing model on the candidate set. 

## 4 | Selection and Results

As mentioned above, we select a random forest model that includes the following hyperparameters:
- *Number of trees*: 100 trees
- *Maximum depth of trees*: 50% of features
- *Minimum observations per leaf*: 5 samples per leaf.
- *Percent of observations sampled*: 75% of observations resampled.

This selection makes sense to us, as it attempts to use a fair amount of data but controls overfitting by reducing the number of features and averaging over a larger forest among hyperparameter options. It's also not surprising to use that regression forests performed well. Compared to KNN and penalized multinomial regression, a regression forest may be more effective at both identifying different groupings of households in poverty and avoiding overfitting to a local neighborhood due to the averaging across many trees.

### 4.1 | Cross-validation accuracy

When trained on the cross-validation set training sample, the performance is as follows:

| Type | F1 Score (Min) | F1 Score (Mean) | F1 Score (Max) |
|----|----|----|----|
| Random Forest | 35.1% | 38.2% | 40.5% |
| KNN | 32.1% | 34.1% | 35.8% |
| Penalized Multinomial Regression | 7.1% | 26.5% | 37.8% |

The random forest training seemed to prefer models with larger numbers of trees and fewer samples per leaf. There were no solid patterns for the number of samples used and the maximum depth of the tree. This matches our expectation that overfitting is a concern, given the variance in the data. Similarly, the KNN model preferred larger numbers of neighbors but did not show a pattern for weighted or unweighted performance.

Conversely, the penalized regression model did not perform as we expected, heavily preferring an $L_1$ penalty and weaker penalization ($\lambda$ = 0.01), which would lead to a more complicated model. The more penalized models performed much worse than random chance.

### 4.2 | Test set accuracy

Finally, we retrain the random forest classifier with the hyperparameters we tested on the entire training data (complete with oversampling from 2,390 households to 6,244). Then, we test the model on a set that we set aside prior to 598 households. The performance is as follows:

| Type | % of Test Sample | Accuracy | Precision | Recall | F1 Score |
|------|------------------|----------|-----------|--------|----------|
| **Overall** | - | **61.7%** | - | - | **36.6%** |
| Non-vulnerable Households | 65.9% | - | 79.8% | 82.5% | 81.1% |
| Vulnerable Households | 10.2% | - | 10.1% | 9.8% | 10.0% |
| Moderate Poverty | 15.2% | - | 27.1% | 28.6% | 27.8% |
| Extreme Poverty | 8.7% | - | 33.3% | 23.1% | 27.2% |

Although we have good overall accuracy, we notably fail to effectively predict vulnerable households, with an F1 rate of about 10 percent. However, for impoverished households, our F1 scores are above random chance on the resampled training data. Notably, the recall scores are worse than the precision scores for our Vulnerable and Extreme Poverty classes, which would be most relevant for the policy context.

Our ROC curve and confusion matrix for the model show similar results, with our predictions being by far the strongest for the largest category, "Non-vulnerable households," and just outperforming random chance for the poverty categories:

<a href="url" style="display: inline-block"><img src='../assets/rf_conf_matrix.png' align="center"></a>
<a href="url" style="display: inline-block"><img src='../assets/rf_roc.png' align="center" height = '75%'></a>

## 5 | Conclusion

We attempted to develop a working PMT for Costa Rican households based on limited household survey data. Our resulting model was more accurate than random chance by about 10 percent on a class-wise basis but heavily overpredicted our most frequent category, even with oversampling-based methods included in our training. Of further concern is that the model did not have solid recall performance for the labels of interest: households in poverty that could receive social programs. This limits how useful the final model could be in the policy context it would be used for.

Our reflections on the process are primarily focused on the data. Based on the Kaggle results and our experience, these data proxy a very limited set of information to understand household income. Beyond the obvious connection between wealth and income, very few variables would plausibly have explanatory details on local variation in poverty. Although our process could no doubt be improved with more time, one of our key conclusions is that this is a complex problem with the given data. Additional observable data on household behaviors and productive assets could likely be included to improve the model's performance.

If this project were to be continued, we would explore different approaches to training the model by using expanded parameters. Furthermore, we would likely conduct our hyperparameter search more intentionally, such as by choosing hyperparameters through gradient descent or exploring promising models in more detail after a first attempt at exploring the models. Beyond having a more comprehensive search across hyperparameters, we would also like to consider different approaches to oversampling these models. One approach could be to convert this process into an ensemble model with one stage to predict the "Non-vulnerable households" as a dichotomous label, which we can do with a reasonably high degree of accuracy, and then predict the minority classes from the subset of data points not classified as "Non-vulnerable households."